# ECE/CS 434 | MP2: DUET
<br />
<nav>
    <span class="alert alert-block alert-warning">Due on Wednesday Feb 21 11:59PM on Gradescope</span>
   
</nav><br> 


## Objective
In this MP, you will:
- Implement DUET algorithm to separate a mixture of N voice signals from received from two microphones

---
## Problem Overview
Consider a problem of separating N sources ($S_1$, $S_2$, ... $S_N$) from recordings on 2 microphones ($R_1$ and $R_2$).
According to DUET algorithm, you will need to perform the following steps:

- Calculate the short-time Fourier transform of two received signals to get the time-frequency spectrograms
- Calculate the ratio of the two time-frequency spectrograms to get relative delay and attenuation
- Cluster the time-frequency bins in the 2D space spanned by relative delay and attenuation
- Recover the original N signals based on the clustering results

You can refer to the original DUET paper in ICASSP 2000: "Blind separation of disjoint orthogonal signals: demixing N sources from 2 mixtures" and this tutorial in Blind speech separation, 2007 - Springer: "The DUET blind source separation algorithm"

For the sake of easier clustering, the exact number of sources N will be provided to you.

You can assume there is no time-frequency bin collision for any two sources.

---
## Imports & Setup
To run the grading script of this MP, you will need to install the Python [SpeechRecognition](https://pypi.org/project/SpeechRecognition/) package. The SpeechRecognition package also requires the dependency [pocketsphinx](https://pypi.org/project/pocketsphinx/). You may directly use pip install to install both packages.
The following `code` cell, when run, imports the libraries you might need for this MP. Feel free to delete or import other commonly used libraries. Double check with the TA if you are unsure if a library is supported.

In [1]:
import numpy as np
import scipy.io.wavfile
import speech_recognition as sr

if __name__ == '__main__':
    import matplotlib.pyplot as plt
    # plt.style.use("seaborn") # This sets the matplotlib color scheme to something more soothing
    from IPython import get_ipython
    get_ipython().run_line_magic('matplotlib', 'inline')

# This function is used to format test results. You don't need to touch it.
def display_table(data):
    from IPython.display import HTML, display
    html = "<table>"
    for row in data:
        html += "<tr>"
        for field in row:
            html += "<td><h4>%s</h4><td>"%(field)
        html += "</tr>"
    html += "</table>"
    display(HTML(html))

### Sanity-check

Running the following code block verifies that the correct module versions are indeed being used. 

Try restarting the Python kernel (or Jupyter) if there is a mismatch even after intalling the correct version. This might happen because Python's `import` statement does not reload already-loaded modules even if they are updated.

In [2]:
if __name__ == '__main__':
    from IPython.display import display, HTML

    def printc(text, color):
        display(HTML("<text style='color:{};weight:700;'>{}</text>".format(color, text)))

    _requirements = [r.split("==") for r in open(
        "packages.txt", "r").read().split("\n")]

    import sys
    for (module, expected_version) in _requirements:
        try:
            if sys.modules[module].__version__ != expected_version:
                printc("[✕] {} version should to be {}, but {} is installed.".format(
                    module, expected_version, sys.modules[module].__version__), "#f44336")
            else:
                printc("[✓] {} version {} is correct.".format(
                    module, expected_version), "#4caf50")
        except:
            printc("[–] {} is not imported, skipping version check.".format(
                module), "#03a9f4")

---
## Your Implementation
Implement your localization algorithm in the function `duet_source_separation(mic_data_folder, NUM_SOURCES)`. Do **NOT** change its function signature. You are, however, free to define and use helper functions. 

Your implementation for `duet_source_separation` function should **NOT** output any plots or data. It should only return the user's calculated location.

In [3]:
def duet_source_separation(mic_data_folder, NUM_SOURCES):
    """DUET source separation algorithm. Write your code here.

    Args:
        mic_data_folder: name of folder (without a trailing slash) containing 
                         two mic datafiles `0.wav` and `1.wav`.

    Returns:
        NUM_SOURCES * recording_length numpy array, where NUM_SOURCES is the number of sources,
        and recording_length is the original length of the recording (in number of samples)

    """

    

    # Step 1: Take STFT of both mics bin by bin 
    


    # to ensure .wav files are playable the output is first converted to a numpy int16 array, before being written to a .wav file
    output = np.int16(output)
    scipy.io.wavfile.write("temp.wav", 22050, output)
    
    return output

In [31]:
# Test Cell

"""
Simply put we are solving for s within the following:

x = As

x represents an array of the audio files we are given
A represents our AoA matrix, which we will solve for with the DUET algorithm
s represents the data of the voices after being seperated, this is what we are trying to solve for
"""

# Step 1: Take STFT of both mics bin by bin 

import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile

sample_rate, samples = wavfile.read('dataset1/0.wav')
# frequencies, times, spectrogram = signal.spectrogram(samples, sample_rate)

sample_rate1, samples1 = wavfile.read('dataset1/1.wav')
# frequencies1, times1, spectrogram1 = signal.spectrogram(samples1, sample_rate1)

from scipy.signal import ShortTimeFFT
from scipy.signal.windows import gaussian

T_x, N = 1 / sample_rate, len(samples)  # 20 Hz sampling rate for 50 s signal
t_x = np.arange(N) * T_x  # time indexes for signal
f_i = 1 * np.arctan((t_x - t_x[N // 2]) / 2) + 5  # varying frequency
x = np.sin(2*np.pi*np.cumsum(f_i)*T_x) # the signal

g_std = 8  # standard deviation for Gaussian window in samples
w = gaussian(50, std=g_std, sym=True)  # symmetric Gaussian window
SFT = ShortTimeFFT(w, hop=10, fs=sample_rate, mfft=200, scale_to='magnitude')
Sx = SFT.stft(samples)  # perform the STFT

Sx1 = SFT.stft(samples1)  # perform the STFT

# Calculate the phase difference by dividing the values
deltaPhi = Sx / Sx1

fig1, ax1 = plt.subplots(figsize=(6., 4.))  # enlarge plot a bit
t_lo, t_hi = SFT.extent(N)[:2]  # time range of plot
ax1.set_title(rf"STFT ({SFT.m_num*SFT.T:g}$\,s$ Gaussian window, " +
              rf"$\sigma_t={g_std*SFT.T}\,$s)")
ax1.set(xlabel=f"Time $t$ in seconds ({SFT.p_num(N)} slices, " +
               rf"$\Delta t = {SFT.delta_t:g}\,$s)",
        ylabel=f"Freq. $f$ in Hz ({SFT.f_pts} bins, " +
               rf"$\Delta f = {SFT.delta_f:g}\,$Hz)",
        xlim=(t_lo, t_hi))

im1 = ax1.imshow(abs(Sx), origin='lower', aspect='auto',
                 extent=SFT.extent(N), cmap='viridis')
ax1.plot(t_x, f_i, 'r--', alpha=.5, label='$f_i(t)$')
fig1.colorbar(im1, label="Magnitude $|S_x(t, f)|$")

# Shade areas where window slices stick out to the side:
for t0_, t1_ in [(t_lo, SFT.lower_border_end[0] * SFT.T),
                 (SFT.upper_border_begin(N)[0] * SFT.T, t_hi)]:
    ax1.axvspan(t0_, t1_, color='w', linewidth=0, alpha=.2)
for t_ in [0, N * SFT.T]:  # mark signal borders with vertical line:
    ax1.axvline(t_, color='y', linestyle='--', alpha=0.5)
ax1.legend()
fig1.tight_layout()
plt.show()





# deltaPhi = spectrogram / spectrogram1

# fig = plt.figure()
# ax = fig.gca(projection='3d')

# ax.plot_surface(frequencies[:, None], times[None, :], 10.0*np.log10(deltaPhi))
# plt.show()


# plt.pcolormesh(times, frequencies, deltaPhi)
# plt.imshow(spectrogram)
# plt.ylabel('Frequency [Hz]')
# plt.xlabel('Time [sec]')
# plt.show()

# plt.pcolormesh(times1, frequencies1, np.log(deltaPhi))
# #plt.imshow(spectrogram)
# plt.ylabel('Frequency [Hz]')
# plt.xlabel('Time [sec]')
# plt.show()

# Plotting Histogram of deltaPhi
# print(deltaPhi.shape)

# deltaPhi_flat = deltaPhi.flatten()

# print(np.std(deltaPhi_flat))

# print(deltaPhi_flat[:20])

# #Plotting a basic histogram
# tf_bins = len(times) * len(frequencies)
# plt.hist(deltaPhi_flat, bins=tf_bins, color='skyblue', edgecolor='black')
 
# # Adding labels and title
# plt.xlabel('Values')
# plt.ylabel('Frequency')
# plt.title('Basic Histogram')
 
# # Display the plot
# plt.show()

ImportError: cannot import name 'ShortTimeFFT' from 'scipy.signal' (/opt/homebrew/Caskroom/miniconda/base/envs/ECE434_MP2_env/lib/python3.9/site-packages/scipy/signal/__init__.py)

---
## Running and Testing
Use the cell below to run and test your code, and to get an estimate of your grade.

In [5]:
def calculate_score(calculated, expected):
    student_result = set()
    calculated = np.array(calculated)
    if calculated.shape[0] != len(expected):
      return 0, {'Incorrect number of sources!'}
    for i in range(calculated.shape[0]):
        scipy.io.wavfile.write("temp.wav",22050,calculated[i,:])
        r = sr.Recognizer()
        with sr.AudioFile("temp.wav") as source:
            audio = r.record(source)
        try:
            text = r.recognize_sphinx(audio)
            student_result.add(text.lower())
        except:
            student_result.add("Sphinx could not understand audio")
    score = len(student_result.intersection(expected))/len(expected)
    return score, student_result
     
if __name__ == '__main__':
    groundtruth = [{"hello how are you"}, {"nice to meet you","how are you"}, {"how are you","good morning","nice to meet you"}]
    
    output = [['Dataset', 'Expected Output', 'Your Output', 'Grade', 'Points Awarded']]
    for i in range(1,4):
        directory_name = 'dataset{}'.format(i)
        student_output = duet_source_separation(directory_name, i)
        result = calculate_score(student_output, groundtruth[i-1])   
        output.append([
            str(i),
            str(groundtruth[i-1]), 
            str(result[1]), 
            "{:2.2f}%".format(result[0] * 100),
            "{:1.2f} / 5.0".format(result[0] * 5),
        ])

    output.append([
        '<i>👻 Hidden test 1 👻</i>', 
        '<i>???</i>', 
        '<i>???</i>', 
        '<i>???</i>', 
        "<i>???</i> / 10.0"])
    output.append([
        '<i>...</i>', 
        '<i>...</i>', 
        '<i>...</i>', 
        '<i>...</i>', 
        "<i>...</i>"])
    output.append([
        '<i>👻 Hidden test 7 👻</i>', 
        '<i>???</i>', 
        '<i>???</i>', 
        '<i>???</i>', 
        "<i>???</i> / 10.0"])
    display_table(output)

UnboundLocalError: local variable 'output' referenced before assignment

---
## Rubric
You will be graded on the three data points provided to you (5 points each) and seven additional data points under different settings(10 points each). We will use the same code from the **Running and Testing** section above to grade all 10 traces of data. We will run ASR on your output to see if it generates the corrected separated speech signal. Output order does not matter. Percentage of grade for each data point is based on how many sources you estimated correctly (i.e., assume there are n sources, then you will get $\frac{1}{n} * 100\%$ for each correctedly estimated source).

---
## Submission Guidlines
This Jupyter notebook (`MP2.ipynb`) is the only file you need to submit on Gradescope. As mentioned earlier, you will only be graded using your implementation of the `duet_source_separation` function, which should only return the calculated **NOT** output any plots or data. 

**Make sure any code you added to this notebook, except for import statements, is either in a function or guarded by `__main__`(which won't be run by the autograder). Gradescope will give you immediate feedback using the provided test cases. It is your responsibility to check the output before the deadline to ensure your submission runs with the autograder.**